In [ ]:
%load_ext etl_visual_explained
import pandas as pd
import numpy as np

## 1. Generazione di un Dataset di Test
Creiamo un dataset fittizio con vendite e spese per simulare un ambiente Retail.

In [ ]:
np.random.seed(42)

dates = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")
stores = ["Milano", "Roma", "Napoli", "Torino"]
categories = ["Elettronica", "Abbigliamento", "Alimentari"]

# Creazione raw data
df_raw = pd.DataFrame({
    "date": np.random.choice(dates, 1000),
    "store": np.random.choice(stores, 1000),
    "category": np.random.choice(categories, 1000),
    "sales": np.random.uniform(50, 500, 1000),
    "expenses": np.random.uniform(20, 200, 1000),
    "returned_items": np.random.choice([0, 1, np.nan], 1000, p=[0.8, 0.1, 0.1])
})

df_raw.head()

## 2. Test 1: Filtraggio e Pulizia di Base
Testiamo come il plugin gestisce metodi come dropna(), query() e assign().
Guarda nell'UI come variano le righe e vengono valutati i valori nulli.

In [ ]:
%%visual_chain

clean_df = (
    df_raw
    .dropna(subset=["returned_items"])
    .query("sales > 100")
    .assign(net_profit=lambda x: x["sales"] - x["expenses"])
)

## 3. Test 2: Aggregazione Complessa
La `.groupby` è uno degli step più ostici di pandas da debuggare visivamente.
Vediamo come si comporta il tracciamento dimensionale.

In [ ]:
%%visual_chain

store_performance = (
    clean_df
    .groupby(["store", "category"])
    .agg(
        total_sales=("sales", "sum"),
        avg_profit=("net_profit", "mean"),
        total_returns=("returned_items", "sum")
    )
    .reset_index()
    .sort_values("total_sales", ascending=False)
)